In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from pathlib import Path
import time


from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 
from selenium.webdriver.support.select import Select

url = 'https://www.nba.com/stats/players/traditional?SeasonType=IST'
xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[2]/div[1]/div[3]/div/label/div/select'
def get_ptables(url_list,path_list):
    data = []
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)
    cookie_check = False
    for i in range(len(url_list)):
        url = url_list[i]
        xpath = path_list[i]
        print(url)
        
        driver.get(url)
        accept_path = '//*[@id="onetrust-accept-btn-handler"]'
        time.sleep(5)

        if EC.presence_of_element_located((By.XPATH, accept_path)) and cookie_check == False:
            driver.find_element(By.XPATH, accept_path).click() 
            cookie_check = True
            time.sleep(1)
        

        element = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, xpath)))
        # Wait for the page to fully load
        #time.sleep(5)
        '''if check_exists_by_xpath(driver, "//a[contains(text(),'>')]/preceding-sibling::a[1]"):
            number_of_pages = int(driver.find_element(By.XPATH, "//a[contains(text(),'>')]/preceding-sibling::a[1]").text)
            print(number_of_pages)'''
        
        dropdown1 = Select(driver.find_element(By.XPATH, xpath))
        dropdown1.select_by_index(0)

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')
        

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))
        #needed table is at the end
        df= dfs[-1]

       
        data.append(df)
    driver.close()
    return data
urls = [url]
paths = [xpath]
data = get_ptables(urls,paths)
df = data[0]
df = df[[col for col in df.columns if ' RANK' not in col]]
df = df.drop(columns=['Unnamed: 0'])
df.to_csv('inseason_2024.csv',index = False)

https://www.nba.com/stats/players/traditional?SeasonType=IST


In [6]:
def get_table(year,minutes,ps = False):
    if ps == False:
        stype = 'leagues'
    else:
        stype='playoffs'
    link_1 = 'https://www.basketball-reference.com/'+stype+'/NBA_'+str(year)+'_totals.html'
    df = pd.read_html(link_1)[0]
   
    df = df[df["MP"].notna()]
    df = df[df['MP'] != 'MP']
    df['MP'] = df['MP'].astype(float)
    df['PTS'] = df['PTS'].astype(float)
    df['FTA'] = df['FTA'].astype(float)
    df['FGA'] = df['FGA'].astype(float)

    df['TS%'] = df['PTS']/(2* (df['FGA'] + .44 *df['FTA'] ))

    df = df[df['MP'] >minutes]
    df['TS%'] *=100
    df['G'] = df['G'].astype(int)
    
    #print(df)
    return [ df[['Player','TS%','PTS','MP','Tm','G']],year]
def get_table2(year,minutes,ps = False):
    if ps == False:
        stype = 'leagues'
    else:
        stype='playoffs'
    link_1 = 'https://www.basketball-reference.com/'+stype+'/NBA_'+str(year)+'_per_poss.html'
    df = pd.read_html(link_1)[0]
   
    df = df[df["MP"].notna()]
    df = df[df['MP'] != 'MP']
    df['MP'] = df['MP'].astype(float)
    df['PTS'] = df['PTS'].astype(float)
    df['FTA'] = df['FTA'].astype(float)
    df['FGA'] = df['FGA'].astype(float)

    df['TS%'] = df['PTS']/(2* (df['FGA'] + .44 *df['FTA'] ))

    df = df[df['MP'] >minutes]
    df['TS%'] *=100
    df['G'] = df['G'].astype(int)
    
    #print(df)
    return [ df[['Player','TS%','PTS','MP','Tm','G','FTA','FGA']],year]
year = 2024
minutes = 0
ps = False
if ps == False:
    df = pd.read_csv('scoring.csv')
    df = df[df.year<year]
    new_table,year = get_table(year,minutes,ps)
    df = pd.concat([df,new_table])
    df.to_csv('scoring.csv',index=False)
    print(df)
    
    df = pd.read_csv('totals.csv')
    df = df[df.year<year]
    new_table,year = get_table2(year,minutes,ps)
    new_table['year'] = year
    df = pd.concat([df,new_table])
    df.to_csv('totals.csv',index = False)
    
elif ps == True:
    df = pd.read_csv('scoring_ps.csv')
    df = df[df.year<year]
    new_table = get_table(year,minutes,ps)
    df = pd.concat([df,new_table])
    df.to_csv('scoring_ps.csv',index=False)
    
    df = pd.read_csv('totals_ps.csv')
    df = df[df.year<year]
    new_table = get_table2(year,minutes,ps)
    df = pd.concat([df,new_table])
    df.to_csv('totals_ps.csv',index= False)
df

                   Player        TS%    PTS      MP   Tm   G    year
0         Zaid Abdul-Aziz  51.502980   15.9  2459.0  HOU  79  1974.0
1    Kareem Abdul-Jabbar*  56.254004   28.1  3548.0  MIL  81  1974.0
2               Don Adams  45.660701   14.9  2298.0  DET  74  1974.0
3           Rick Adelman*  44.808816   13.5   618.0  CHI  55  1974.0
4            Lucius Allen  53.568266   24.2  2388.0  MIL  72  1974.0
..                    ...        ...    ...     ...  ...  ..     ...
481        Thaddeus Young  51.546392    4.0     6.0  TOR   2     NaN
482            Trae Young  53.099054  220.0   322.0  ATL   9     NaN
483        Omer Yurtseven  73.529412   16.0    31.0  UTA   6     NaN
484           Cody Zeller  25.933610    5.0    35.0  NOP   6     NaN
485           Ivica Zubac  61.971831   88.0   201.0  LAC   8     NaN

[25475 rows x 7 columns]


,Player,TS%,PTS,MP,Tm,G,FTA,FGA,year
0,Zaid Abdul-Aziz,51.635626,865.0,2459.0,HOU,79,240.0,732.0,1974
1,Kareem Abdul-Jabbar*,56.358679,2191.0,3548.0,MIL,81,420.0,1759.0,1974
2,Don Adams,45.698666,759.0,2298.0,DET,74,201.0,742.0,1974
3,Rick Adelman*,44.730633,182.0,618.0,CHI,55,76.0,170.0,1974
4,Lucius Allen,53.612502,1268.0,2388.0,MIL,72,274.0,1062.0,1974
...,...,...,...,...,...,...,...,...,...
481,Thaddeus Young,51.628025,32.6,6.0,TOR,2,16.3,24.4,2024
482,Trae Young,53.110523,32.1,322.0,ATL,9,13.0,24.5,2024
483,Omer Yurtseven,73.371510,24.6,31.0,UTA,6,3.1,15.4,2024
484,Cody Zeller,26.120533,6.9,35.0,NOP,6,8.2,9.6,2024


In [10]:
start_year = 1974
end_year = 2024
averages = pd.read_html('https://www.basketball-reference.com/leagues/NBA_stats_per_game.html#stats', header=1)[0]

#averages= averages.dropna()

averages = averages[averages['Season']!='Season']
averages = averages[averages['PTS']!='Per Game']

averages['PTS'] = averages['PTS'].astype(float)
averages['FGA'] = averages['FGA'].astype(float)
averages['FTA'] = averages['FTA'].astype(float)
averages['TS%'] = averages['PTS']/(2* (averages['FGA'] + .44 *averages['FTA'] ))
averages['TS%'] = averages['PTS']/(2* (averages['FGA'] + .44 *averages['FTA'] ))
averages = averages[['TS%','Season']]
averages['Season'] = averages['Season'].str[:4]
averages['Season'] = averages['Season'].astype(int)
averages['Season']+=1
averages = averages[averages['Season']>=start_year]
averages = averages[averages['Season']<=end_year]
averages = averages.iloc[::-1]

In [12]:
averages.to_csv('tsavg.csv',index = False)